# How do you know if your results are correct? #
When you query the Lahman or Statcast database, or any large database for that matter, it's easy to get a result and just assume it's correct. Systematically verifying that your results are correct is more difficult. But, it's an essential part of any data analysis task.

There are numerous resources on data validation and data verification and how to develop tests to ensure that you don't have errors in your results. What I'm presenting here is a bit more of a hackier method to use, and is primarly based on issues I saw in the first homework.

## Process ##
1. Check your approach to the problem
    * Check definitions - make sure you're understanding of terms like ERA, OBP, SLG, FIP are correct before building an argument around them. Look up the definitions and depending on the work you're producing, you might want to reference a source. 
    * Check that you're using the correct metrics - if you want to evaluate pitching, use a pitching stat, for example. Read articles or talk to other people to get a broader understanding of the context in which different metrics are used.
    * Check that you understand the task
2. Verify that your code is correct
    * When you generate a dataframe, each row should be unique by at least one parameter. For example, if your data frame is a list of players and teams since 2010, there should be either a unique playerID, year, or team in each row. If you have multiple rows with exactly the same data, you have a bug in your merge code.    
3. Verify that the data is correct
    * The data in the dataframe should be both correct and complete, which means that every row in the dataframe should be there and that the dataframe isn't missing any data.
    * If you have a large dataframe, it can help to save it in a csv and then spot check the csv. 
    * Select a few players in the csv and compare them to a known reference. For example, if you're generating a csv of players with a BA average below 0.200, you can look for those players on Baseball Reference and verify that their BA is below 0.200 and that their team and year match your data.
    * Select a few players from Baseball Reference with a BA below 0.200 and verify that they are on your csv.
    * Select players to cover a range of categories, such as played for one year or multiple years, played for one team or multiple teams, played one position or multiple positions.
4. Verify that your numbers are correct
    * If you're generating a number, such as how many first-pitch strikes did Chad Bettis throw in 2017, verify your number by printing out the individual rows of data. If it's too much data, select a smaller problem, such as how many first-pitch strikes did Chad Bettis throw in April 2017. 
5. Make the data smaller
    * If you just have too much data to work with, find a way to limit your query and generate a smaller dataframe and perform verification on that. 
    
# Location, location, location #
### wOBA ###
In the previous lecture, we calculated the run value of different events using the RE before and after the event and the number of runs that the event scored. We generated the following run values.

| Event | Run Val |
|---|---|
| HR | 1.397 |
| 3B | 1.070 |
| 2B | 0.776 |
| E | 0.508 |
| 1B | 0.475 |
| HBP | 0.352 |
| BB | 0.323 |
| IBB | 0.179 |
| SB | 0.175 |
| Sac Bunt | -0.096 |
| Out | -0.299 |
| CS | -0.467 |

We can use these run values to calculate wOBA, which is weighted on-base average. It measures a player's overall offensive contributions per plate appearance. 

To calculate wOBA, the weights of each event are scaled relative to an out. An out is worth 0 and all other values are shifted up by the weight of an out. For example, instead of a HR having a weight of about 1.4, it has a weight of 1.7. 

The run values are then scaled by 1.15 to put the overall wOBA calculation on the same scale as OBP, which is about 0.320 league average.

$$wOBA=\frac{0.72*NIBB + 0.75*HBP + 0.89*1B + 1.24*2B + 1.57*3B +
1.95*HR}{AB + NIBB + SF + HBP}$$
## Ballparks matter ##
Baseball teams play 162 games a year. They play almost everyday from April through September. Half of their games are played at home, in the same stadium, and half of their games are played on the road at different stadiums. 
#### What is a series? ####
When teams play each other, they don't play just one game. They typically play three games, but they might play two games or four games. That group of games is called a series. Teams do well when they win a series by winning the majority of the games. For example, in a three game series, a team tries to win at least two games. If they win all three, that's even better. Each individual game contributes to a team's overall record.

#### Hitters and pitchers parks ####
In every ballpark, the bases are 90 ft apart and the pitchers mound is 60ft 6in from home plate. However, the dimensions of the outfield are different at every ballpark.

Some ballparks are neutral. The dimensions of the outfield are considered average. There is an average number of homeruns hit. The BA, OBP, SLG at the ballpark match the league average for those values. Pitchers have an average ERA in the innings pitched there. 

Neutral Parks
* Fenway Park in Boston, MA
* Dodger Stadium in Los Angeles, CA

Some ballparks are hitters parks, which means that they are more favorable to hitters than pitchers. It might be due to small outfield dimensions. Or, the outfield dimensions might be average, but the air is thin or hot and humid and the ball flies further when hit. Hitters parks typically give up more homeruns than average parks. 

Hitters Parks
* Coors Field in Denver
* Camden Yard in Baltimore
* Great American Ballpark in Cincinnati

Other parks have some aspect that makes them a hitters park for some hitters. Fenway Park in Boston has a particularly short right field, which favors left-handed hitters. 

Parks can also be hitters parks due to a small foul territory. Foul balls that would have been caught for outs end up in the stands instead and the batter gets another chance.

Some ballparks are pitchers parks, which means that they are more favorable to pitchers than hitters. Often, the air is heavy in pitchers parks. The park might be close to the water where the air is humid and cold.

Pitchers Parks
* Petco Park in San Diego
* Oracle Park in San Francisco

#### Offensive and defensive stats vary by ballpark ####
The fact that the ballpark matters so much is unique to baseball. In basketball, for example, the court is the same size everywhere. In football, the weather can be an issue, but the field is always the same size. 

In trying to evaluate how good a player is, we need to consider the ballpark where he plays half of his games. We can determine how a park plays by looking at the runs scored at the park and comparing to runs scored at other parks.

### Park factors ###
Park factors are an attempt to normalize performance across ballparks by stripping out the effect of the ballpark. It's not a perfect science. It's an approximation. 

A park factor is a number that represents how much more, or less, offense is produced at the park than at an average ballpark.

The league average park factor = 100. A park factor of 110 means that offense is elevated $10\%$ over average. To consider park factors in wOBA, a player's stats would be decreased $5\%$ reflecting that they play half their games in that environment and half their games at league average.

#### Calculating park factors ####
There are different types of park factors, and just like everything, park factors are continuing to evolve.

**Run park factors**
* How does the number of runs scored per game compare to other parks

**Event park factors**
* How does park affect behavior of events. e.g. number of homeruns or BABIP

**Performance park factors**
* How does park affect player performance metrics, such as BA

##### One approach to calculating park factors #####
* Example: BA park factor
* In one year at Coors Field, Rockies and opponents had 1,752 hits in 5,739 at-bats in 81 games.
* COL home 1,752/5,739 = 0.305 BA, for all players at Coors that year
* In same year in 81 road games, there were 1,413 hits in 5,467 at-bats.
* COL away 1,413/5,467 = 0.258 BA, for Rockies players and the teams they played against

$$PF_{BA}=\frac{0.305}{0.258}=1.18$$

This means that BA is elevated $18\%$ at Coors Field compared to the average at the other parks where COL plays. You could also say that the PF is 118 if we use the scale of 100 as average.

**BABIP**

BABIP in COL = 0.339
BABIP away = 0.293
$$PF_{BABIP}=\frac{0.339}{0.293} = 1.157$$

Examples for Coors Field for BABIP, Runs per game, HR per game

**Plate appearances per game**
PA per game in COL = 40.5
PA per game away = 37.9
$$PF_{PA}=\frac{40.5}{37.9} = 1.067$$

**Number of pitches per game**
NP per game in COL = 143.9
NP per game away = 135.4
$$PF_{PA}=\frac{40.5}{37.9} = 1.063$$

#### Reading on park factors ####
https://library.fangraphs.com/the-beginners-guide-to-understanding-park-factors/

## Marginal Payroll/Marginal Wins ##
You can think Marginal Payroll/Marginal Wins (MP/MW) as how well does your team do compared to how well they would do if you spent the minimum money you had to. Think of this as baseline performance for your algorithm. MP/MW evaluates how efficiently a front office spends money to acquire talented players for the team. The team's payroll for the year and its record is compared to the record it could expect from fielding a roster of replacement-level players, all of whom are paid the major league minimum salary. 

The formula is:
$$MPMW=\frac{club\_payroll - 28*league\_min}{WP-0.300 * 162}$$

where $league\_min$ is the minimum league salary that year, and $WP$ is the team's winning percentage. The formula assumes that a replacement-level club would play 0.300 ball. 

The team's winning percentage above 0.300 is multiplied by 162 to calculate the number of marginal wins over a full 162-game season.

The club payroll assumes a 25-man active roster and three-man disabled list, and uses the Opening Day payroll numbers as the best measure of a team's expectations for the season. MPMW
basically measures how much a team had to pay for a win. There are different categories that a team can fall into regarding their MPMW score.

* Low MP/MW, good record: Efficient ballclub (2003 Marlins, Athletics)
* Low MP/MW, bad record: Not spending enough to compete (2003 Devil Rays)
* High MP/MW, good record: Spending its way to the top (2003 Yankees)
* High MP/MW, bad record: Poorly-run club (2003 Mets, Rangers)

In 2017, the league min salary was $\$535,000.$ The Rockies club payroll was $\$127.8M$. Their winning percentage was $\$87/162 = 0.54.$